In [1]:
import time
import adodbapi
import pandas as pd
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [9]:
client_manuf =["Malard Nicolas"]
client_brands = ["Nicolas"]

decimals = 2
sign = "after"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

prodORitem="SKU"

categories =["Effervescents"]
sectors = ["Champagnes","Mousseux","Effervescents Sans Alcool"]
segments = ["Blanc De Blancs","Bruts","Crus","Extra Brut","Millesimes","Rosés","Mousseux Bourgogne","Mousseux Italiens","Mousseux Val De Loire","Mousseux Alsace","Mousseux Bordelais"]
subsegments= []
subcategories= []

national = True
customareas= "Revenue"
areas = ["NATIONAL","CHANNEL","REGION", f'{customareas}']
 
regions_RET  = []
channels_RET = []
market_RET = []
 
regions_CHAN = ["NICOLAS", "HMSM"]
channels_CHAN = ["NICOLAS QCN","NICOLAS VCN","NICOLAS QCT","NICOLAS QCA","NICOLAS CCP"]
market_CHAN = []

regions_REG = []
channels_REG = ["NICOLAS IDF","NICOLAS PAC","NICOLAS RHO","NICOLAS AQU","NICOLAS EST"]
market_REG = []

regions_CUST = []
channels_CUST = ["CA A","CA B","CA C","CA D","CA E"]
market_CUST = []
 
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending June 2025"
end_date = "2025-07-01"

years = {2023,2024,2025}

ManufOrTopC ="Top Companies" #Manufacturer or Top Companies
BrandOrTopB= "Top Brands" #Brand or Top Brands

past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
past_3_months = pd.date_range(end=end_date, periods=3, freq='ME').strftime('%b-%y').tolist()
past_36_months = pd.date_range(end=end_date, periods=36, freq='ME').strftime('%b-%y').tolist()

National= ["NATIONAL"] if national else []
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST
brands_only = True  # Get the Data of SKU Share by brands level only


entity_hierarchy = [
    ("Area","NATIONAL",National),
    ("Region", "RETAILER", regions_RET),
    ("Channel", "RETAILER", channels_RET),
    ("Market", "RETAILER",  market_RET),
    ("Region", "CHANNEL",regions_CHAN),
    ("Channel", "CHANNEL",channels_CHAN),
    ("Market", "CHANNEL",market_CHAN),
   ("Region", "REGION",regions_REG),
   ("Channel","REGION",channels_REG),
   ("Market","REGION", market_REG),
   ("Region",customareas, regions_CUST),
    ("Channel", customareas, channels_CUST),
    ("Market", customareas, market_CUST)
]
print(entity_hierarchy)
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
 
]
direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}

server = "powerbi://api.powerbi.com/v1.0/myorg/Nicolas"
dataset_name = "Nicolas Sparkling Dataset"

p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"
p3m_dax = "{" + ", ".join(f'"{date}"' for date in past_3_months) + "}"
p36m_dax = "{" + ", ".join(f'"{date}"' for date in past_36_months) + "}"

print(p12m_dax)
path=os.path.join(os.getcwd(),"Pricing Datasets NewEX")

conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=900;"
print(conn_str)

[('Area', 'NATIONAL', ['NATIONAL']), ('Region', 'RETAILER', []), ('Channel', 'RETAILER', []), ('Market', 'RETAILER', []), ('Region', 'CHANNEL', ['NICOLAS', 'HMSM']), ('Channel', 'CHANNEL', ['NICOLAS QCN', 'NICOLAS VCN', 'NICOLAS QCT', 'NICOLAS QCA', 'NICOLAS CCP']), ('Market', 'CHANNEL', []), ('Region', 'REGION', []), ('Channel', 'REGION', ['NICOLAS IDF', 'NICOLAS PAC', 'NICOLAS RHO', 'NICOLAS AQU', 'NICOLAS EST']), ('Market', 'REGION', []), ('Region', 'Revenue', []), ('Channel', 'Revenue', ['CA A', 'CA B', 'CA C', 'CA D', 'CA E']), ('Market', 'Revenue', [])]
{"Jul-24", "Aug-24", "Sep-24", "Oct-24", "Nov-24", "Dec-24", "Jan-25", "Feb-25", "Mar-25", "Apr-25", "May-25", "Jun-25"}
Provider=MSOLAP.8;Data Source=powerbi://api.powerbi.com/v1.0/myorg/Nicolas;Initial Catalog=Nicolas Sparkling Dataset;Timeout=900;


## Price Positioning

In [3]:
def execute_dax_query(BrandorManuf,entity_name,entity_type, mkt, area, hierby):
    outputdic = {}
    key =  f"{entity_type} | {entity_name}"


    columns = [
        "Relative Price",'Av Price/Unit','Value Sales','Value Share','Value Share DYA','Av Price/KG','IYA Price/KG'
    ]
    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{BrandorManuf}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Grand total query (not grouped by BrandOrTopB, just Category)
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            

        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            grand_tot[df.columns[0]] = 'Grand Total'

            # Reorder columns if necessary
            grand_tot = grand_tot[df.columns]
            
            

            # Concatenate the two

            df = pd.concat([df, grand_tot], ignore_index=True)
            outputdic[key] = df  

            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")

    return outputdic



def process_dax_queries(BrandorManuf,entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{value} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query,BrandorManuf, entity,value,mkt, area, hierby)
                            futures[future] = key
       
        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        if BrandorManuf==f'{BrandOrTopB}':
            filename =  f"price_positioning_brands.pkl"
        else:
            filename =  f"price_positioning_manuf.pkl"
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")

process_dax_queries(f'{BrandOrTopB}',entity_hierarchy,hierarchy_levels) 
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy,hierarchy_levels) 

Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS VCN.
Query executed successfully for National.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS RHO.
Query executed successfully for NICOLAS AQU.
Query executed successfully for NICOLAS EST.
Query executed successfully for CA A.
Query executed successfully for CA C.
Query executed successfully for CA B.
Query executed successfully for CA D.
Query executed successfully for CA E.
Query executed successfully for NICOLAS.
Query executed successfully for National.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS VCN.
Query executed successfully 

## Sector/Segment/SubSegment Leaderships (SINGLE)

In [11]:
def execute_dax_query(entity_name, mkt, area, hierby, client, total=True):
    outputdic = {}
    key = f"{client} | {entity_name}" if client else f"{entity_name}"

    
    columns = [
        "Value Share", "Value Sales", "Av Price/KG", "WoB %", "Gross Margin %"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Prepare client brand filter conditionally
    client_filter = ""
    if not total and client:
        client_filter = f'''
            FILTER(
                Products,
                Products[{BrandOrTopB}] = "{client}"
            ),
        '''

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            TREATAS({{"{area}"}}, Market[Area]),
            TREATAS({{"{entity_name}"}}, Market[{mkt}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()


        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        grand_tot[df.columns[0]] = 'Grand Total'
        grand_tot = grand_tot[df.columns]
        df = pd.concat([df, grand_tot], ignore_index=True)
        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None, total=False):
    with ThreadPoolExecutor(max_workers=8) as executor:
        
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                if client_brands: 
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            for client in client_brands:
                                key = f"{client} | {entity}"
                                ordered_keys.append(key)
                                future = executor.submit(
                                    execute_dax_query, entity, mkt, area, hierby, client, total
                                )
                                futures[future] = key
                else:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{entity}"
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query, entity, mkt, area, hierby, '', total
                            )
                            futures[future] = key

            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            if client_brands:
                filename = f"{hierby}_leadership.pkl"
            else:
                filename = f"{hierby}_total_leadership.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

# Execute
if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands)

process_dax_queries(entity_hierarchy, hierarchy_levels, total=True)


Query executed successfully for HMSM.
Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS VCN.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS AQU.
Query executed successfully for NICOLAS EST.
Query executed successfully for NICOLAS RHO.
Query executed successfully for CA B.
Query executed successfully for CA C.
Query executed successfully for CA A.
Query executed successfully for CA D.
Query executed successfully for CA E.
All DataFrames for Category saved to c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\Category_leadership.pkl.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NA

## Sector/Segment Leaderships

In [12]:
def execute_dax_query(entity_name, mkt, area, hierby,direct_parent, client,manuf, total=False):
    outputdic = {}
    key = f"{client} | {entity_name}" if client else (f"{manuf} | {entity_name}" if manuf else f"{entity_name}")


    columns = [
        "Value Share", "Value Sales", "Av Price/KG", "WoB %", "Gross Margin %"
    ]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Prepare client brand filter conditionally
    client_filter = ""
    if not total and client:
        client_filter = f'''
            FILTER(
                Products,
                Products[{BrandOrTopB}] = "{client}"
            ),
        '''    
    manuf_filter=''
    if not total and manuf:
        manuf_filter = f'''
            FILTER(
                Products,
                Products[{ManufOrTopC}] = "{manuf}"
            ),
        '''

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}],
                    Products[{hierby}]
                    
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{direct_parent[hierby]}]                   
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
        # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            TREATAS({{"{area}"}}, Market[Area]),
            TREATAS({{"{entity_name}"}}, Market[{mkt}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"

        if maintotal_df.empty:
            outputdic[key] = maintotal_df
            return outputdic
        if not maintotal_df.empty:
            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
    
        if not grand_tot.empty:
            # Create a dict for the first two columns
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan  # or pd.NA

            # Ensure all required columns exist, fill missing ones with NaN
            for col in df.columns:
                if col not in grand_tot.columns:
                    grand_tot[col] = np.nan

            # Reorder columns exactly as in df
            grand_tot = grand_tot[df.columns]
            df = pd.concat([df_with_totals, grand_tot], ignore_index=True)

        outputdic[key] = df

        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, client_brands=None,client_manuf=None, total=False):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys = []
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category" or direct_parent[hierby]=="Category":
                    continue
            if isinstance(hier_values, list):
                if client_brands and not total: 
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            for client in client_brands:
                                key = f"{client} | {entity}"
                                ordered_keys.append(key)
                                future = executor.submit(
                                    execute_dax_query, entity, mkt, area, hierby,direct_parent, client,'', ''
                                )
                                futures[future] = key
                if client_manuf and not total:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            for manuf in client_manuf:
                                key = f"{manuf} | {entity}"
                                ordered_keys.append(key)
                                future = executor.submit(
                                    execute_dax_query, entity, mkt, area, hierby,direct_parent, '',manuf,''
                                )
                                futures[future] = key               
                elif total==True:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{entity}"
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query, entity, mkt, area, hierby, direct_parent,'','', total
                            )
                            futures[future] = key

            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)

            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]

            if client_brands:
                filename = f"brand_{hierby}.pkl"
            elif client_manuf:
                filename = f"manuf_{hierby}.pkl"
            else:
                filename = f"total_{hierby}.pkl"

            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            
            print(f"All DataFrames for {hierby} saved to {output_file}.")

if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, client_brands=client_brands)
if client_manuf:
    process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, client_manuf=client_manuf)   
process_dax_queries(entity_hierarchy, hierarchy_levels,direct_parent, total=True)


Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS VCN.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS RHO.
Query executed successfully for CA B.
Query executed successfully for NICOLAS EST.
Query executed successfully for CA A.
Query executed successfully for NICOLAS AQU.
Query executed successfully for CA C.
Query executed successfully for CA D.
Query executed successfully for CA E.
All DataFrames for Segment saved to c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\brand_Segment.pkl.
Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOL

## Shelf & Avg Price / KG By Brand

In [13]:
def execute_dax_query(manuforbrand,entity_name,entity_type, mkt, area, hierby, incall=True):
    outputdic = {}
    key =  f"{entity_type} | {entity_name}"
    
    if manuforbrand==f'{BrandOrTopB}':
        columns = [
            "Relative Price", "Av Price/Unit", "Value Sales", "IYA Price/KG", "Base Price/KG","Av Price/KG","Value Share","WoB %","Value Share DYA"
        ]
    else:
        columns = ["Base Price/KG","Av Price/KG","Value Share","WoB %","Value Share DYA"]
 

    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
 
    # Prepare client brand filter conditionally
    scope_filter = ""
    if incall:
        scope_filter = f'''
            FILTER('Scope', 'Scope'[Scope] = "Category"),
            Products[{direct_parent[hierby]}] = "{entity_type}"
            
        '''
    else:
        scope_filter = f'''
            FILTER('Scope', 'Scope'[Scope] = "{hierby}"),
            Products[{direct_parent[hierby]}] = "{entity_type}"
        '''
 
    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}],
                    Products[{manuforbrand}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{"{area}"}}, Market[Area]),
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            {scope_filter}
        )
    """
    parenttotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}]
 
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            {scope_filter}
        )
    """
   
    # Grand total query
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                VALUES(Products[Category]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            {scope_filter}
 
        )
    """
 
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns_result = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
            
 
        df = pd.DataFrame(data, columns=columns_result)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
 
 
        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]


        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]

 
        if maintotal_df.shape[1] > 1:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
 
        if maintotal_df.empty:
            outputdic[key] = maintotal_df
            return outputdic
        if not maintotal_df.empty:
            df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
 
        if not grand_tot.empty:
            # Ensure 'Grand Total' label is added to the first column of grand_tot
            grand_tot[df_with_totals.columns[0]] = 'Grand Total'
 
            # Keep only columns that exist in df_with_totals
            common_columns = [col for col in df_with_totals.columns if col in grand_tot.columns]
            grand_tot = grand_tot[common_columns]
 
            # Align grand_tot with df_with_totals in case column order matters
            grand_tot = grand_tot.reindex(columns=df_with_totals.columns)



        df = pd.concat([df_with_totals, grand_tot], ignore_index=True)
        outputdic[key] = df
 
 
        print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")
 
    return outputdic
 
 
def process_dax_queries(manuforbrand,entity_hierarchy, hierarchy_levels, incall=False):
    with ThreadPoolExecutor(max_workers=8) as executor:
        hierarchy_dict = dict(hierarchy_levels)
 
        for hierby, _ in hierarchy_levels:
            if hierby == "Category":
                continue
                   
            dfs_results = {}
            futures = {}
            ordered_keys = []
            # Get the parent level name (e.g., 'Segment')
            parent_level = direct_parent[hierby]
            parent_values = hierarchy_dict.get(parent_level, [])
            if isinstance(parent_values, list):
                for value in parent_values:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            key = f"{value} | {entity}"                          
                            ordered_keys.append(key)
                            future = executor.submit(
                                execute_dax_query,manuforbrand, entity, value, mkt, area, hierby,incall
                            )
                            futures[future] = key
            temp_results = {}
            for future in as_completed(futures):
                result = future.result()
                temp_results.update(result)
 
            for key in ordered_keys:
                if key in temp_results:
                    dfs_results[key] = temp_results[key]
            if manuforbrand==f'{BrandOrTopB}':
                if incall==False:
                    filename = f"shelf_{hierby}_top_brands.pkl"
                else:
                    filename = f"shelf_{hierby}_all_brands.pkl"
            else:
                if incall==False:
                    filename = f"shelf_{hierby}_top_manuf.pkl"
                else:
                    filename = f"shelf_{hierby}_all_manuf.pkl"
 
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
           
            print(f"All DataFrames for {hierby} saved to {output_file}.")
 
 
 
# Execute
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels,incall=False)
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels, incall=True)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,incall=False)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels, incall=True)

Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS VCN.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS.
Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS AQU.
Query executed successfully for NICOLAS RHO.
Query executed successfully for NICOLAS EST.
Query executed successfully for CA C.
Query executed successfully for CA B.
Query executed successfully for CA A.
Query executed successfully for CA D.
Query executed successfully for CA E.
All DataFrames for Sector saved to c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\shelf_Sector_top_brands.pkl.
Query executed successfully for NICOLAS QCA.
Query executed successfully for 

## Price Point Distribution By Product (Item)


In [14]:
def execute_dax_query(entity_name, entity_type, mkt, area, hierby, client, manuf,p12m=True):
    outputdic = {}

    key = f"{client} | {entity_name} | {entity_type}" if client else f"{manuf} | {entity_name} | {entity_type}" 
    
    if p12m :
        timeper=p12m_dax
    else:
        timeper=p3m_dax    
    columns = ["Base Price/Unit", "Base Price/KG", "Value Sales", "Gross Margin %"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    client_filter=''
    if client:
        client_filter = f'''
                FILTER(
                    Products,
                    Products[{BrandOrTopB}] = "{client}" &&
                    Products[{ManufOrTopC}] = "{manuf}"
                ),
            '''
   
    manuf_filter = f'FILTER(Products, Products[{ManufOrTopC}] = "{manuf}"),' if manuf else ""

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{hierby}],
                    Products[{prodORitem}],
                    Products[Total Size]
                ),
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """

    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{hierby}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    itemtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products,Products[{prodORitem}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            {client_filter}
            {manuf_filter}
            TREATAS({{"{area}"}}, Market[Area]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}])
        )
    """

    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

            with conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
          
            with conn.cursor() as cursor:
                cursor.execute(itemtotal_dax_query)
                itemtotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])


            with conn.cursor() as cursor:
                cursor.execute(grandtotal_query)
                grand_tot = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

        for dataframe in [df,maintotal_df,itemtotal_df, grand_tot]:
            dataframe.columns = dataframe.columns.str.replace(r'.*\[|\]', '', regex=True)
            dataframe.dropna(how='all', inplace=True)

        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, maintotal_df], ignore_index=True)
      
        if not itemtotal_df.empty:
            itemtotal_df.iloc[:, 0] = itemtotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, itemtotal_df], ignore_index=True)
            
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)

        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None, client_manuf=None,p12m=True):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue

            hierarchy_dict = dict(hierarchy_levels)
            parent_level = direct_parent.get(hierby)
            parent_values = hierarchy_dict.get(parent_level, [])

            if not parent_values:
                continue

            futures = {}
            ordered_keys = []

            # Determine prodORitem per level

            for value in parent_values:
                for mkt, area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        if client_brands:
                            for client in client_brands:
                                for manuf in client_manuf:
                                    key = f"{client} | {entity} | {value}"
                                    ordered_keys.append(key)
                                    futures[executor.submit(
                                        execute_dax_query, entity, value, mkt, area, hierby, client, manuf,p12m
                                    )] = key
                        elif client_manuf:
                            for manuf in client_manuf:
                                key = f"{manuf} | {entity} | {value}"
                                ordered_keys.append(key)
                                futures[executor.submit(
                                    execute_dax_query, entity, value, mkt, area, hierby, None, manuf, p12m
                                )] = key

            dfs_results = {}
            for future in as_completed(futures):
                result = future.result()
                dfs_results.update(result)

            dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}
            if p12m:
                 filename = f"price_distribution_{hierby}_{'p12m' if client_brands else 'manuf_p12m'}.pkl"
            else:
                 filename = f"price_distribution_{hierby}_{'p3m' if client_brands else 'manuf_p3m'}.pkl"
                     
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            print(f"All DataFrames for {hierby} saved to {output_file}.")


# Example Invocation
if client_brands:
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands,client_manuf=client_manuf,p12m=True)
    process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands,client_manuf=client_manuf,p12m=False)

process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf,p12m=True)
process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf,p12m=False)




Query executed successfully for HMSM.
Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS VCN.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS RHO.
Query executed successfully for NICOLAS AQU.
Query executed successfully for CA A.
Query executed successfully for NICOLAS EST.
Query executed successfully for CA B.
Query executed successfully for CA C.
Query executed successfully for CA E.
Query executed successfully for CA D.
All DataFrames for Sector saved to c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_Sector_p12m.pkl.
Query executed successfully for HMSM.
Query executed successfully for 

C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)
C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)


Query executed successfully for CA E.
Query executed successfully for NICOLAS.


C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)
C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)
C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with emp

Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS VCN.
Query executed successfully for NICOLAS QCT.


...

C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)


Query executed successfully for NICOLAS EST.


C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)
C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)
C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with emp

Query executed successfully for CA A.
Query executed successfully for CA B.
Query executed successfully for CA C.
Query executed successfully for CA D.
Query executed successfully for CA E.
All DataFrames for SubCategory saved to c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_SubCategory_manuf_p3m.pkl.


C:\Users\SophieZIMMERMANN\AppData\Local\Temp\ipykernel_32336\738960321.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, maintotal_df], ignore_index=True)


## Price Point Distribution By Product Scraped(Item)

In [8]:
# def execute_dax_query(entity_name, mkt, hierby, client, manuf):
#     outputdic={}
#     if not categories or len(categories) == 0:
#         raise ValueError("categories list must have at least one element")

#     columns = [
#         "Scraped Av. Price/Unit", "Scraped Av. Price/KG"
#     ]
#     column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

#     client_filter = ''
#     if client:
#         client_filter = f'FILTER(Products, Products[{BrandOrTopB}] = "{client}")'

#     manuf_filter = ''
#     if manuf:
#         manuf_filter = f'FILTER(Products, Products[Top Companies] = "{manuf}")'

#     # Compose CALCULATETABLE filter arguments list (only non-empty filters)
#     filter_args = [f'FILTER(Products, Products[Category] = "{categories[0]}")']
#     if client_filter:
#         filter_args.append(client_filter)
#     if manuf_filter:
#         filter_args.append(manuf_filter)
#     filter_args.append(f'TREATAS({{ "{entity_name}" }}, Market[{mkt}])')
#     filter_args.append('FILTER(\'Scope\', \'Scope\'[Scope] = "Category")')

#     filters_str = ",\n            ".join(filter_args)

#     dax_query = f"""
#         EVALUATE
#         VAR BaseTable =
#             CROSSJOIN(
#                 VALUES(Calendar[End of Week]),
#                 VALUES(Products[{prodORitem}]),
#                 VALUES(Products[Total Size])
#             )
#         VAR FilteredTable =
#             CALCULATETABLE(
#                 BaseTable,
#                 {filters_str}
#             )
#         RETURN
#             ADDCOLUMNS(
#                 FilteredTable,
#                 {column_exprs}
#             )
#         """

#     parenttotal_dax_query = f"""
#             EVALUATE
#             CALCULATETABLE(
#                 ADDCOLUMNS(
#                     SUMMARIZECOLUMNS(
#                         Products[{hierby}]

#                     ),
#                 {column_exprs}
#             ),
#             Products[Category] = "{categories[0]}",
#             {client_filter}
#             {manuf_filter}
#             TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
#             FILTER('Scope', 'Scope'[Scope] = "Category")
#         )
#     """
#         # Grand total query
#     grandtotal_query = f"""
#         EVALUATE
#         CALCULATETABLE(
#             ADDCOLUMNS(
#                 VALUES(Products[Category]),
#                 {column_exprs}
#             ),
#             Products[Category] = "{categories[0]}",
#             {client_filter}
#             {manuf_filter}
#             TREATAS({{ "{entity_name}" }}, Market[{mkt}])
#         )
#     """

#     try:
#         with adodbapi.connect(conn_str) as conn:
#             conn.timeout = 500  # Increase timeout to 300 seconds
#             with conn.cursor() as cursor:
#                 cursor.execute(dax_query)
#                 columns_result = [desc[0] for desc in cursor.description]
#                 data = cursor.fetchall()
                
#         with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
#             cursor.execute(parenttotal_dax_query)
#             maintotal_columns = [desc[0] for desc in cursor.description]
#             maintotal_data = cursor.fetchall()
            
#         with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
#             cursor.execute(grandtotal_query)
#             grandtotal_columns = [desc[0] for desc in cursor.description]
#             grandtotal_data = cursor.fetchall()

#         df = pd.DataFrame(data, columns=columns_result)
#         df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
#         df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
#         maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
#         maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
#         maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
#         grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
#         grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
#         grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        
#         if maintotal_df.shape[1] > 1:
#             maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"

#         if maintotal_df.empty:
#             outputdic[key] = maintotal_df
#             return outputdic
#         if not maintotal_df.empty:
#             df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
    
#         if not grand_tot.empty:
#             # Create a dict for the first two columns
#             grand_tot[df.columns[0]] = 'Grand Total'
#             grand_tot[df.columns[1]] = np.nan  # or pd.NA

#             # Ensure all required columns exist, fill missing ones with NaN
#             for col in df.columns:
#                 if col not in grand_tot.columns:
#                     grand_tot[col] = np.nan

#             # Reorder columns exactly as in df
#             grand_tot = grand_tot[df.columns]
#             df = pd.concat([df_with_totals, grand_tot], ignore_index=True)

#         outputdic[key] = df

#         print(f"Query executed successfully for {entity_name}.")
#     except adodbapi.DatabaseError as db_error:
#         print(f"Database error for {entity_name} in {mkt}: {db_error}")
#     except Exception as e:
#         print(f"Unexpected error for {entity_name} in {mkt}: {e}")

#     return outputdic


# def process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=None,client_manuf=None):
#     with ThreadPoolExecutor(max_workers=8) as executor:
#         dfs_results = {} 
#         futures = {}
#         ordered_keys = []
#         for hierby, hier_values in hierarchy_levels:
#             if hierby == "Category":
#                     continue
#             if isinstance(hier_values, list):
#                 if client_brands: 
#                     for mkt, entity_list in entity_hierarchy:
#                         for entity in entity_list:
#                             for manuf in client_manuf:
#                                 for client in client_brands: 
#                                     key = f"{client} | {entity}"
#                                     ordered_keys.append(key)
#                                     future = executor.submit(
#                                         execute_dax_query, entity, mkt, hierby, client,manuf
#                                     )
#                                     futures[future] = key
#                 else:
#                     for mkt, entity_list in entity_hierarchy:
#                         for entity in entity_list:
#                             for manuf in client_manuf:
#                                 key = f"{manuf} | {entity}"
#                                 ordered_keys.append(key)
#                                 future = executor.submit(
#                                     execute_dax_query, entity, mkt, hierby, '',manuf
#                                 )
#                                 futures[future] = key               


#             temp_results = {}
#             for future in as_completed(futures):
#                 result = future.result()
#                 temp_results.update(result)

#             for key in ordered_keys:
#                 if key in temp_results:
#                     dfs_results[key] = temp_results[key]

#             if client_brands:
#                 filename = f"price_distribution_scraped_{hierby}.pkl"
#             else:
#                 filename = f"price_distribution_scraped_{hierby}_manuf.pkl"


#             output_file = f"{path}\\{filename}"
#             with open(output_file, "wb") as f:
#                 pd.to_pickle(dfs_results, f)
            
#             print(f"All DataFrames for {hierby} saved to {output_file}.")

# if client_brands:
#     process_dax_queries(entity_hierarchy, hierarchy_levels, client_brands=client_brands,client_manuf=client_manuf)
# if client_manuf:
#     process_dax_queries(entity_hierarchy, hierarchy_levels, client_manuf=client_manuf)   


## Price Point Distribution By Brand

In [15]:
def execute_dax_query(entity_name, entity_type, mkt, area, hierby):
    outputdic = {}

    key = f"{entity_name} | {entity_type}" 
    


    columns = "Av Price/Unit", "Value Share"
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{BrandOrTopB}],
                    Products[Pack Size]
                ),
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{BrandOrTopB}]),
                {column_exprs}
            ),
            Products[{hierby}] = "{entity_type}",
        
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            Products[{hierby}] = "{entity_type}",
            
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")

        )
    """

    try:
        # 1. Main query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            data_columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()

        # 2. Parent total query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(parenttotal_dax_query)
            maintotal_columns = [desc[0] for desc in cursor.description]
            maintotal_data = cursor.fetchall()

        # 3. Grand total query
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()

        # Now build DataFrames safely
        df = pd.DataFrame(data, columns=data_columns)
        df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
        df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]

        maintotal_df = pd.DataFrame(maintotal_data, columns=maintotal_columns)
        maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
        maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        

        grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
        grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
        grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]
        



        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df,maintotal_df], ignore_index=True)
  
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)

        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")

    return outputdic


def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{entity} | {value}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query, entity,value,mkt, area, hierby)
                            futures[future] = key
       
        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        filename = f"price_distribution_by_brands_category.pkl"
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")

process_dax_queries(entity_hierarchy,hierarchy_levels) 



Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS VCN.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS RHO.
Query executed successfully for NICOLAS AQU.
Query executed successfully for NICOLAS EST.
Query executed successfully for CA A.
Query executed successfully for CA B.
Query executed successfully for CA C.
Query executed successfully for CA D.
Query executed successfully for CA E.
Query executed successfully for NATIONAL.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS VCN.
Query executed successfully for NICOLAS QCT.
Query executed successfully 

## By brand For Sec,Seg,SubSeg... (Slide 7)


In [ ]:
def execute_dax_query(entity_name, entity_type, mkt, area, hierby):
    outputdic = {}
 
    key = f"{entity_name} | {entity_type}"
    
    columns = "Av Price/Unit", "Value Share"
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
             
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[{BrandOrTopB}],
                    Products[{hierby}],
                    Products[Pack Size]
                ),
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{direct_parent[hierby]}")
        )
    """
 
    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{BrandOrTopB}]),
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    itemtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products,Products[{BrandOrTopB}],
                Products[{hierby}]),
                {column_exprs}
            ),
            Products[{direct_parent[hierby]}] = "{entity_type}",
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
 
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            Products[Category] = "{categories[0]}",
            TREATAS({p12m_dax}, Calendar[MonthYear]),
            TREATAS({{"{area}"}}, Market[Area]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}])
        )
    """
 
    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
 
            with conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
         
            with conn.cursor() as cursor:
                cursor.execute(itemtotal_dax_query)
                itemtotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
 
 
            with conn.cursor() as cursor:
                cursor.execute(grandtotal_query)
                grand_tot = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
 
        for dataframe in [df,maintotal_df,itemtotal_df, grand_tot]:
            dataframe.columns = dataframe.columns.str.replace(r'.*\[|\]', '', regex=True)
            dataframe.dropna(how='all', inplace=True)
       
            cols_to_check = ["Av Price/Unit", "Value Share"]
            dataframe.drop(
                index=dataframe[
                    (dataframe[cols_to_check].abs() < 1e-6).all(axis=1)
                ].index,
                inplace=True
            )    
           
 
        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, maintotal_df], ignore_index=True)
     
        if not itemtotal_df.empty:
            itemtotal_df.iloc[:, 1] = itemtotal_df.iloc[:, 1].astype(str) + " Total"
            df = pd.concat([df, itemtotal_df], ignore_index=True)
           
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)
 
        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")
 
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")
 
    return outputdic
 
 
def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        for hierby, hier_values in hierarchy_levels:
            if hierby == "Category":
                continue
 
            hierarchy_dict = dict(hierarchy_levels)
            parent_level = direct_parent.get(hierby)
            parent_values = hierarchy_dict.get(parent_level, [])
 
            if not parent_values:
                continue
 
            futures = {}
            ordered_keys = []
 
            # Determine prodORitem per level
 
            for value in parent_values:
                for mkt, area, entity_list in entity_hierarchy:
                    for entity in entity_list:
                        for manuf in client_manuf:
                            key = f"{entity} | {value}"
                            ordered_keys.append(key)
                            futures[executor.submit(
                                execute_dax_query, entity, value, mkt, area, hierby
                            )] = key
 
            dfs_results = {}
            for future in as_completed(futures):
                result = future.result()
                dfs_results.update(result)
 
            dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}
           
            filename = f"price_distribution_by_brands_{hierby}"
                     
            output_file = f"{path}\\{filename}"
            with open(output_file, "wb") as f:
                pd.to_pickle(dfs_results, f)
            print(f"All DataFrames for {hierby} saved to {output_file}.")
 
 
process_dax_queries(entity_hierarchy, hierarchy_levels)
 
 
 

Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS VCN.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS CCP.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS AQU.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS RHO.
Query executed successfully for NICOLAS EST.Query executed successfully for CA A.

Query executed successfully for CA B.
Query executed successfully for CA C.
Query executed successfully for CA D.
Query executed successfully for CA E.
All DataFrames for Sector saved to c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing slide duplicate\Pricing Datasets NewEX\price_distribution_by_brands_Sector.
Query executed successfully for NATIONAL.
Query executed successfully

## Price Point Comparison by Product

In [ ]:
def execute_dax_query(ManuforBrand,entity_name, entity_type, mkt, area, hierby, p12m=True):
    outputdic = {}

    key = f"{entity_name} | {entity_type}"
    
    if p12m:
        timeper = p12m_dax
        columns = ["Value Share", "Gross Margin %"]
        row_fields = [
            f"Products[{ManuforBrand}]",
            f"Products[{prodORitem}]"
        ]
    else:
        timeper = p3m_dax
        columns = ["Base Price/Unit", "Base Price/KG", "Value Sales", "Value Share"]
        row_fields = [
            f"Products[{ManuforBrand}]",
            f"Products[{prodORitem}]",
            "Products[Total Size]"
        ]

    summarize_fields = ",\n                        ".join(row_fields)
    column_exprs = ",\n                ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    {summarize_fields}
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            Products[{hierby}] = "{entity_type}",
            TREATAS({timeper}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    # Parent total query
    parenttotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products, Products[{ManuforBrand}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            Products[{hierby}] = "{entity_type}",
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """
    itemtotal_dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(Products,Products[{prodORitem}]),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({timeper}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )
    """

    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE( 
            ADDCOLUMNS(VALUES(Products[Category]), {column_exprs}),
            TREATAS({timeper}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            TREATAS({{ "{entity_name}" }}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
            
        )
    """

    try:
        
        
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

            with conn.cursor() as cursor:
                cursor.execute(parenttotal_dax_query)
                maintotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
          
            with conn.cursor() as cursor:
                cursor.execute(itemtotal_dax_query)
                itemtotal_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])


            with conn.cursor() as cursor:
                cursor.execute(grandtotal_query)
                grand_tot = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
          
            numeric_columns = columns
            for i, dataframe in enumerate([df, maintotal_df, itemtotal_df, grand_tot]):
                dataframe.columns = dataframe.columns.str.replace(r'.*\[|\]', '', regex=True)
                dataframe.dropna(how='all', inplace=True)

                if not dataframe.empty:
                    # Force conversion to numeric in case some values are strings
                    dataframe[numeric_columns] = dataframe[numeric_columns].apply(pd.to_numeric, errors='coerce')

                    # Now drop rows where all numeric values are 0.0
                    mask_all_zero = (dataframe[numeric_columns].fillna(0.0) == 0.0).all(axis=1)
                    dataframe = dataframe[~mask_all_zero]

                if i == 0:
                    df = dataframe
                elif i == 1:
                    maintotal_df = dataframe
                elif i == 2:
                    itemtotal_df = dataframe
                else:
                    grand_tot = dataframe



        if not maintotal_df.empty:
            maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            df = pd.concat([df, maintotal_df], ignore_index=True)
            
        if not p12m:
            if not itemtotal_df.empty:
                itemtotal_df.iloc[:, 0] = itemtotal_df.iloc[:, 0].astype(str) + " Total"
                df = pd.concat([df, itemtotal_df], ignore_index=True)
            
        if not grand_tot.empty:
            grand_tot[df.columns[0]] = 'Grand Total'
            grand_tot[df.columns[1]] = np.nan
            grand_tot = grand_tot.reindex(columns=df.columns)
            df = pd.concat([df, grand_tot], ignore_index=True)

        outputdic[key] = df
        print(f"Query executed successfully for {entity_name}.")

    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")

    return outputdic


def process_dax_queries(ManuforBrand,entity_hierarchy, hierarchy_levels,p12m):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{entity} | {value}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query,ManuforBrand, entity,value,mkt, area, hierby,p12m)
                            futures[future] = key

        dfs_results = {}
        for future in as_completed(futures):
            result = future.result()
            dfs_results.update(result)

        dfs_results = {key: dfs_results[key] for key in ordered_keys if key in dfs_results}
        if ManuforBrand ==f'{BrandOrTopB}' :
            if p12m:
                    filename = f"price_point_by_brands_items_P12M.pkl"
            else:
                    filename = f"price_point_by_brands_items_P3M.pkl"
        else:
            if p12m:
                    filename = f"price_point_by_manuf_items_P12M.pkl"
            else:
                    filename = f"price_point_by_manuf_items_P3M.pkl"           
        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        print(f"All DataFrames for {hierby} saved to {output_file}.")


# Example Invocation
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels,p12m=True)
process_dax_queries(f'{BrandOrTopB}',entity_hierarchy, hierarchy_levels,p12m=False)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,p12m=True)
process_dax_queries(f'{ManufOrTopC}',entity_hierarchy, hierarchy_levels,p12m=False)



Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS VCN.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS PAC.
Query executed successfully for NICOLAS RHO.
Query executed successfully for NICOLAS EST.
Query executed successfully for CA A.
Query executed successfully for NICOLAS AQU.
Query executed successfully for CA B.
Query executed successfully for CA C.
Query executed successfully for NATIONAL.
Query executed successfully for CA D.
Query executed successfully for CA E.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS QCT.
Query executed successfully 

## Price Correlation

In [ ]:
def execute_dax_query(entity_name,entity_type, mkt, area, hierby):
    outputdic = {}
    key =  f"{entity_type} | {entity_name}"
    

    columns = [
        "Volume Share",'Av Price/KG','Value Share'
    ]
    
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)

    # Main query
    dax_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                CROSSJOIN(
                    DISTINCT(Products[{BrandOrTopB}]),
                    DISTINCT(Calendar[QuarterStart]),
                    DISTINCT(Calendar[End of Week])
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )

    """
    fristcoltot_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    products,
                    Products[{BrandOrTopB}]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "{hierby}")
        )

    """
    secondcoltot_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Calendar,
                    Calendar[QuarterStart]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            Products[{hierby}] = "{entity_type}",
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    # Grand total query (not grouped by BrandOrTopB, just Category)
    grandtotal_query = f"""
        EVALUATE
        CALCULATETABLE(
            ADDCOLUMNS(
                SUMMARIZE(
                    Products,
                    Products[Category]
                ),
                {column_exprs}
            ),
            Products[Category] = "{categories[0]}",
            TREATAS({p36m_dax}, Calendar[MonthYear]),
            TREATAS({{"{entity_name}"}}, Market[{mkt}]),
            TREATAS({{"{area}"}}, Market[Area]),
            FILTER('Scope', 'Scope'[Scope] = "Category")
        )
    """
    try:
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(dax_query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(fristcoltot_query)
            fristttotal_columns = [desc[0] for desc in cursor.description]
            fristtotal_data = cursor.fetchall()       
             
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(secondcoltot_query)
            sectotal_columns = [desc[0] for desc in cursor.description]
            sectotal_data = cursor.fetchall()  
        with adodbapi.connect(conn_str) as conn, conn.cursor() as cursor:
            cursor.execute(grandtotal_query)
            grandtotal_columns = [desc[0] for desc in cursor.description]
            grandtotal_data = cursor.fetchall()
            
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)         
            df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows

            maintotal_df = pd.DataFrame(fristtotal_data, columns=fristttotal_columns)
            maintotal_df.columns = maintotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            maintotal_df = maintotal_df.loc[~(maintotal_df.select_dtypes(include='number') == 0).all(axis=1)]
            
            sectotal_df = pd.DataFrame(sectotal_data, columns=sectotal_columns)
            sectotal_df.columns = sectotal_df.columns.str.replace(r'.*\[|\]', '', regex=True)
            sectotal_df = sectotal_df.loc[~(sectotal_df.select_dtypes(include='number') == 0).all(axis=1)]
        
            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]

            if maintotal_df.shape[1] > 1:
                maintotal_df.iloc[:, 0] = maintotal_df.iloc[:, 0].astype(str) + " Total"
            
            if sectotal_df.shape[1] > 1:
                first_col = sectotal_df.columns[0]
                # Ensure datetime
                sectotal_df[first_col] = pd.to_datetime(sectotal_df[first_col], errors='coerce')

                # Format date and append ' total'
                sectotal_df[first_col] = sectotal_df[first_col].dt.strftime('%Y-%m-%d') + " Total"



            if sectotal_df.empty:
                outputdic[key] = sectotal_df
                return outputdic

            if maintotal_df.empty:
                outputdic[key] = maintotal_df
                return outputdic
        
            
            if not maintotal_df.empty:
                df_with_totals = pd.concat([df,maintotal_df], ignore_index=True)
                
            df_with_totals['QuarterStart'] = pd.to_datetime(df_with_totals['QuarterStart']).dt.strftime('%Y-%m-%d')

            if not sectotal_df.empty:
                dfsec_with_totals = pd.concat([df_with_totals,sectotal_df], ignore_index=True)

            

            grand_tot = pd.DataFrame(grandtotal_data, columns=grandtotal_columns)
            grand_tot.columns = grand_tot.columns.str.replace(r'.*\[|\]', '', regex=True)
            grand_tot = grand_tot.loc[~(grand_tot.select_dtypes(include='number') == 0).all(axis=1)]  # Remove zero rows
          
            if not grand_tot.empty:
                # Set first column value to 'Grand Total', second column to NaN
                grand_tot[df.columns[0]] = 'Grand Total'
                grand_tot[df.columns[1]] = np.nan  # or pd.NA

                # Restrict grand_tot to only columns that appear in df
                grand_tot = grand_tot[df.columns.intersection(grand_tot.columns)]

            # Concatenate all together
            df = pd.concat([dfsec_with_totals, grand_tot], ignore_index=True)

            outputdic[key] = df


            
            print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name} in {mkt}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name} in {mkt}: {e}")

    return outputdic



def process_dax_queries(entity_hierarchy, hierarchy_levels):
    with ThreadPoolExecutor(max_workers=8) as executor:
        dfs_results = {} 
        futures = {}
        ordered_keys=[]
        for hierby, hier_values in hierarchy_levels:
            if isinstance(hier_values, list):
                for value in hier_values:
                    for mkt, area, entity_list in entity_hierarchy:
                        for entity in entity_list:
                            # print(hierby,value,entity)                                    
                            key = f"{value} | {entity}"
                            ordered_keys.append(key)
                            future = executor.submit(execute_dax_query, entity,value,mkt, area, hierby)
                            futures[future] = key
       
        temp_results = {}
        for future in as_completed(futures):
            result = future.result()
            temp_results.update(result)

        # Insert results in original order
        for key in ordered_keys:
            if key in temp_results:
                dfs_results[key] = temp_results[key]
        filename =  f"price_correlation_P3Y.pkl"

        output_file = f"{path}\\{filename}"
        with open(output_file, "wb") as f:
            pd.to_pickle(dfs_results, f)
        
        print(f"All DataFrames saved to {output_file}.")

process_dax_queries(entity_hierarchy,hierarchy_levels) 


Query executed successfully for NICOLAS CCP.
Query executed successfully for NICOLAS QCT.
Query executed successfully for NICOLAS QCA.
Query executed successfully for NICOLAS VCN.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS.
Query executed successfully for NATIONAL.
Query executed successfully for NICOLAS QCN.
Query executed successfully for NICOLAS AQU.
Query executed successfully for NICOLAS IDF.
Query executed successfully for NICOLAS RHO.
Query executed successfully for NICOLAS PAC.
Query executed successfully for CA A.
Query executed successfully for NICOLAS EST.
Query executed successfully for CA B.
Query executed successfully for CA C.
Query executed successfully for CA D.
Query executed successfully for NATIONAL.
Query executed successfully for CA E.
Query executed successfully for NICOLAS.
Query executed successfully for HMSM.
Query executed successfully for NICOLAS VCN.
Query executed successfully for NICOLAS QCT.
Query executed successfully 